# Vision-Based Horizon Detection with Dual Global and Local Objectives

The following notebook walks you through an implementation of a computer vision-based horizon detection that uses a two-stage objective that greatly reduces computational overhead compared to the typical exhaustive search approach.

The first objective ("global") attempts to find the range of combinations of "pitch" and "roll" (attitude and angle) corresponding to a halfplane that likely subdivdes the sky from the rest of the image.  The second objective ("local") searches exhaustively through these combinations to find the halfplane that maximizes the difference in average intensity of the two halfplanes in the immediate viscinity of the halfplane.

Compared with an exhaustive search of pitch and roll combinations performed at the outset, this method obtains perfect accuracy on our datset with a full order-of-magnitute less computations.  This method benefits from the assumption that a "sky" as represented by image data has higher intensity values than the ground pixels (higher mean), and that the sky has higher consistency of representation (lower variance).  A "coefficient of variance" calculation (ratio of mean to variance) performed on the full range of angle/attitude for all images in the set suggests that--at least for our data--the optimization surface is approximately convex, allowing for confident use of subsampling in the global objective.

Exhaustive search over the second objective for our dataset in the data exploration phase however reveals numerous local maxima- therefore we must employ exhaustive search over the subregion identified in the global objective in runtime.  Regardless, we observe no loss of accuracy on out output.

The method is demonstrated on a selection of maritime images whose time-of-day, glare effects, and ground-truth horizon location within the frame and varied.

*Main()* and associated functions are located in *two_objectives_horizon_detection.py*.

## Header

In [ ]:
%matplotlib tk

from __future__ import division

import numpy as np
import cv2
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import path
import os

import two_objectives_horizon_detection as tohd

current_dir = os.getcwd()
img_dir = os.path.join(current_dir,"images")
files = [os.path.join("images",x) for x in os.listdir(img_dir) if x.endswith('.jpeg')]

## GLOBAL OBJECTIVE

In [ ]:
#GLOBAL OBJECTIVE SETTINGS
img_file = files[10]
global_img_reduction = 0.1
global_angles = (-90,91,5)
global_distances = (5,100,5) 
global_buffer_size = 3

#GLOBAL OBJECTIVE MAIN ROUTINE
global_search = tohd.main(img_file, 
                          img_reduction = global_img_reduction,
                          angles = global_angles, 
                          distances = global_distances, 
                          buffer_size = global_buffer_size, 
                          local_objective = 0)  #2m5s

#GLOBAL OBJECTIVE OPTIMIZATION SURFACE
objective_1 = np.max((global_search[:,:,0] - global_search[:,:,1]),0) / (global_search[:,:,2])

### Global Objective Optimization Surface

In [ ]:
X = np.arange(0, len(range(*global_angles)), 1)
Y = np.arange(0, len(range(*global_distances)), 1)
X, Y = np.meshgrid(Y, X)
Z = objective_1

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X,Y,Z, rstride=1, cstride=1)
plt.draw()

## LOCAL OBJECTIVE

In [ ]:
#LOCAL OBJECTIVE SETTINGS
above_two_sigma = (2* np.nanstd(objective_1)) + np.nanmean(objective_1)

local_angles = global_search[np.where(objective_1 > above_two_sigma)[0],
                             np.where(objective_1 > above_two_sigma)[1]][:,6]
local_distances = global_search[np.where(objective_1 > above_two_sigma)[0],
                                np.where(objective_1 > above_two_sigma)[1]][:,7]
local_angle_range = (int(np.min(local_angles))-2,
                     int(np.max(local_angles))+3,1)
local_distance_range = (int(np.min(local_distances))-2,
                        int(np.max(local_distances))+3,1)

local_img_reduction = 0.25
local_angles = local_angle_range
local_distances = local_distance_range
local_buffer_size = 5

#LOCAL OBJECTIVE MAIN ROUTINE
print("Evaluating",(len(range(*local_angle_range))*len(range(*local_distance_range))),"candidates...")

local_search = tohd.main(img_file, 
                         img_reduction = local_img_reduction,
                         angles = local_angles, 
                         distances = local_distances, 
                         buffer_size = local_buffer_size, 
                         local_objective = 1)  #2m5s

#LOCAL OBJECTIVE OPTIMIZATION SURFACE
objective_2 =  (local_search[:,:,4] - local_search[:,:,5])**2 / local_search[:,:,2]

#DETECTED HORIZON LINE
horizon_line = local_search[np.unravel_index(objective_2.argmax(), objective_2.shape)]
print("For ",img_file,", best predicted line is",horizon_line[6],"degrees and a distance of",horizon_line[7])

## DETECTED HORIZON LINE

### OVERLAY HORIZON LINE ON INPUT IMAGE AND DISPLAY

In [ ]:
img = cv2.imread(img_file,0)
line_coordinates = tohd.get_plane_indicator_coord(img,int(horizon_line[6]),horizon_line[7]/100,0)[2:4]
cv2.line(img,(line_coordinates[0][0],line_coordinates[0][1]),(line_coordinates[1][0],line_coordinates[1][1]),(0,0,255),2)

plt.imshow(img)
plt.show()